In [ ]:
# Import the general functions for metabolic modelling
%run 'general_functions.ipynb'

glut_off = False # Turn the glutamine constraint off
biomass_off = True # Turn the biomass constraint off
fatty_acid_min = False # Minimize the total fatty acid uptake; otherwise maximum biomass is the objective

growth_norm = True # Results are normalized by the growth rate of each instance 

# Note, leaving both the glutamine and the biomass constraints on will create an infeasible solution as the 
# Oxygen requirement cannot be lowered any further than the pFBA solution

fs = 16 # fontsize for figures

In [ ]:
# model_wt = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_WT.xml")
# model_oxr = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_OxR.xml")
# model_dmso = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_WT+DMSO.xml")
# model_rur = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_RuR+DMSO.xml")

model_wt = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_woBile_WT.xml")
model_oxr = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_woBile_OxR.xml")
model_dmso = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_woBile_WT+DMSO.xml")
model_rur = cobra.io.read_sbml_model("../Models/Colon_Combined_exo_woBile_RuR+DMSO.xml")

models = [model_wt,model_oxr,model_dmso,model_rur]

conditions = ["WT","OxR","WT-DMSO","RuR"]

In [ ]:
# Minimum oxygen requirements of the different conditions before and after growth normalization
o2 = "HMR_9048"
bm = "HCC_biomass"
atpase = "HMR_6916"
pfba_wt_o2 = cobra.flux_analysis.pfba(model_wt,reactions=[o2,bm,atpase])
pfba_oxr_o2 = cobra.flux_analysis.pfba(model_oxr,reactions=[o2,bm,atpase])
pfba_dmso_o2 = cobra.flux_analysis.pfba(model_dmso,reactions=[o2,bm,atpase])
pfba_rur_o2 = cobra.flux_analysis.pfba(model_rur,reactions=[o2,bm,atpase])
pfbas_o2 = [pfba_wt_o2,pfba_oxr_o2,pfba_dmso_o2,pfba_rur_o2]

fva_wt_o2 = flux_variability_analysis(model_wt,[o2,bm,atpase])
fva_oxr_o2 = flux_variability_analysis(model_oxr,[o2,bm,atpase])
fva_dmso_o2 = flux_variability_analysis(model_dmso,[o2,bm,atpase])
fva_rur_o2 = flux_variability_analysis(model_rur,[o2,bm,atpase])
fvas_o2 = [fva_wt_o2,fva_oxr_o2,fva_dmso_o2,fva_rur_o2]


# Set of the results for plotting 
O2_req = [float(pfba[o2]) for pfba in pfbas_o2]
O2_req_lb = [float(fva.loc[o2]['minimum']) for fva in fvas_o2]
O2_req_ub = [float(fva.loc[o2]['maximum']) for fva in fvas_o2]
O2_req_norm = [float(pfba[o2]/pfba[bm]) for pfba in pfbas_o2]
#O2_req_norm = [float(fva.loc[o2]['minimum']/fva.loc[bm]['minimum']) for fva in fvas_o2]
ATP_req = [float(pfba[atpase]) for pfba in pfbas_o2]
ATP_req_lb = [float(fva.loc[atpase]['minimum']) for fva in fvas_o2]
ATP_req_ub = [float(fva.loc[atpase]['maximum']) for fva in fvas_o2]
ATP_req_norm = [float(pfba[atpase]/pfba[bm]) for pfba in pfbas_o2]

# Visualize the results 
matplotlib.rc('xtick', labelsize=fs) 
matplotlib.rc('ytick', labelsize=fs) 
names = ["WT","OxR","WT-DMSO","RuR"]
cols = ["blue","red","black","orange"]

fig, axs = plt.subplots(2,2,figsize=(12,10))

# barWidth = 0.8
# br = np.arange(len(O2_req_lb)) 
# lb = np.array(O2_req_ub)-np.array(O2_req_lb)
# axs[0, 0].bar(br,lb,bottom=O2_req_lb,color=cols,width=barWidth,edgecolor='k',alpha=0.3) 
# axs[0, 0].bar(br,[10]*len(O2_req),bottom=O2_req,color=cols,width=barWidth) 
# axs[0, 0].set_xticks(br)
axs[0, 0].bar(names,O2_req,color=cols,alpha=0.3,edgecolor='black')
axs[0, 0].set_ylim(0,150)
axs[0, 0].set_xticklabels(names,fontsize=fs)
axs[0, 0].set_ylabel("Oxygen Influx",fontsize=fs)
# axs[0, 0].set_xticks([])
axs[0, 0].text(0.98,0.9,"a)", fontsize=fs, verticalalignment='bottom', horizontalalignment='right',transform=axs[0, 0].transAxes)

axs[0, 1].bar(names,O2_req_norm,color="darkgreen",edgecolor='black',alpha=0.2)
axs[0, 1].set_xticklabels(names,fontsize=fs)
axs[0, 1].set_ylabel("Oxygen Influx/ Growth Rate",fontsize=fs)
axs[0, 1].text(0.98,0.9,"b)", fontsize=fs, verticalalignment='bottom', horizontalalignment='right',transform=axs[0, 1].transAxes)
axs[0, 1].set_ylim(0,13)

# lb = np.array(ATP_req_ub)-np.array(ATP_req_lb)
# axs[1, 0].bar(br,lb,bottom=ATP_req_lb,color=cols,width=barWidth,edgecolor='k',alpha=0.3) 
# axs[1, 0].bar(br,[10]*len(ATP_req),bottom=ATP_req,color=cols,width=barWidth) 
# axs[1, 0].set_xticks(br)
axs[1, 0].bar(names,ATP_req,color=cols,alpha=0.3,edgecolor='black')
axs[1, 0].set_ylim(0,150)
axs[1, 0].set_xticklabels(names,fontsize=fs)
axs[1, 0].set_ylabel("Flux through ATPase",fontsize=fs)
# ax1.set_xticks([])
axs[1, 0].text(0.98,0.9,"c)", fontsize=fs, verticalalignment='bottom', horizontalalignment='right',transform=axs[1, 0].transAxes)

axs[1, 1].bar(names,ATP_req_norm,color="darkgreen",edgecolor='black',alpha=0.2)
axs[1, 1].set_xticklabels(names,fontsize=fs)
axs[1, 1].set_ylabel("Flux through ATPase/ Growth Rate",fontsize=fs)
axs[1, 1].text(0.98,0.9,"d)", fontsize=fs, verticalalignment='bottom', horizontalalignment='right',transform=axs[1, 1].transAxes)
axs[1, 1].set_ylim(0,13)

plt.savefig("../Figures/Hypoxia/O2Requirement_woBile.svg")
plt.show()
plt.close()

In [ ]:
# Gather all fatty acid import reactions 
fatty_acid_rxns = [r.id for r in model_wt.reactions if "FAx" in r.id]
offset = 0.0001 # offset not longer required b/c the influx fatty acids are specified
    
if fatty_acid_min:
    # set minimum fatty acid import as the objective 
    for model in models:
        FA_import_constraint = dict()
        for rid in fatty_acid_rxns:
            r = model.reactions.get_by_id(rid)
            FA_import_constraint[r] = 1
        model.objective = FA_import_constraint
        model.objective_direction = 'minimize'
    if growth_norm:
        ymax = 0.25
        xmax = 10
else:
    if growth_norm:
        ymax = 0.7
        xmax = 10
        
if glut_off:
    # Turn off the glutamine influx constraint 
    # which was previously set to experimentally measured values 
    for model in models:
        for rid in ["HMR_9063"]: # glutamine
            r = model.reactions.get_by_id(rid) 
            r.lower_bound = 0.0
            r.upper_bound = 1000.
        
if biomass_off:
    # Turn off the biomass production constraint 
    # which was previously set to experimentally measured values 
    for model in models:
        for rid in ["HCC_biomass"]: 
            r = model.reactions.get_by_id(rid) 
            r.lower_bound = 1.0
            r.upper_bound = 1000.

In [ ]:
o2_fluxes = np.arange(0,160,2)

In [ ]:
# plot production envelope of O2 against glucose/ glutamine
for reac_id,name in zip(["HMR_9063","HMR_9034"],["Gln","Glu"]): # glutamine, glucose 
    plot_o2_pe(reac_id,o2_fluxes,growth_norm=growth_norm)
    #plt.xlim(0,100)
    plt.tight_layout()
    plt.savefig("../Figures/Hypoxia/{}_import_bmOFF{}_GlnOFF{}_FAmin{}_woBile.svg".format(name,biomass_off,glut_off,fatty_acid_min))
    plt.show()
    plt.close()

In [ ]:
cmap = matplotlib.cm.gist_ncar
nsteps = len(fatty_acid_rxns)
cols = [cmap(j/float(nsteps)) for j in range(nsteps)]
random.shuffle(cols)

# plot production envelope of O2 against fatty acids uptake 
for model,condition in zip(models,conditions):
    plt.figure(figsize=(10,6))
    reac_ids = fatty_acid_rxns
    pfba_sols = []
    pfba_all = dict()
    bm_fluxes = []
    for constraint in o2_fluxes:
        ex_o2 = model.reactions.get_by_id("HMR_9048")
        ex_o2.lower_bound = constraint
        ex_o2.upper_bound = constraint
        try:
            bm = model.reactions.get_by_id("HCC_biomass")
            bm_pfba = cobra.flux_analysis.pfba(model,reactions=[bm])
            bm_pfba = bm_pfba[bm.id]
        except:
            bm_pfba = np.nan
        reacs_sum_pfba = 0
        pfba_single = []
        for reac_id in reac_ids:
            r = model.reactions.get_by_id(reac_id)
            try:
                pfba = cobra.flux_analysis.pfba(model,reactions=[r,bm])
                if growth_norm:
                    pfba_result = pfba[reac_id]/bm_pfba
                else:
                    pfba_result = pfba[reac_id]
                reacs_sum_pfba = reacs_sum_pfba + pfba_result
                pfba_single.append(pfba_result)
            except:
                pfba_single.append(np.nan)
        bm_fluxes.append(bm_pfba)
        pfba_sols.append(reacs_sum_pfba)
        pfba_all[constraint] = pfba_single
    if growth_norm:
        o2_fluxes_norm = [o/b for o,b in zip(o2_fluxes,bm_fluxes)]
    line = [0]*len(o2_fluxes)
    new_line = [0]*len(o2_fluxes)
    for j,reac_id in enumerate(reac_ids):
        for z,constraint in enumerate(o2_fluxes):
            new_line[z] = line[z] + pfba_all[constraint][j]
        diffs = [i-j for i,j in zip(new_line,line)]
        max_val = np.nanmax(diffs)
        max_index = diffs.index(max_val)
        if growth_norm:
            o2_fluxes_plot = o2_fluxes_norm
        else:
            o2_fluxes_plot = o2_fluxes
        if max_val > offset:
            r = model.reactions.get_by_id(reac_id)
            met = [m.name for m in r.metabolites][0]
            plt.plot(o2_fluxes_plot,new_line,label=met,color=cols[j],alpha=0.5)
        else:
            plt.plot(o2_fluxes_plot,new_line,label=None,color="gray",alpha=0.5) 
        plt.fill_between(o2_fluxes_plot,line,new_line,color=cols[j],alpha=0.5) 
        line = new_line.copy()
    plt.plot(o2_fluxes_plot,pfba_sols,color='k',label="SUM")
    leg = plt.legend(bbox_to_anchor=(1.05, 1),loc='upper left',fontsize=fs,markerscale=2.)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(8.0)
    plt.xlabel("Oxygen Influx",fontsize=fs)
    plt.ylabel("Fatty Acid Influx",fontsize=fs)
    plt.ylim(0,ymax)
    plt.xlim(0,xmax)
    plt.tight_layout()
    plt.savefig("../Figures/Hypoxia/FA_import_bmOFF{}_GlnOFF{}_FAmin{}_{}_woBile.svg".format(biomass_off,glut_off,fatty_acid_min,condition))
    plt.show()
    plt.close()
        

In [ ]:
exchange_rxns = []
for r in model_wt.reactions:
    if len(r.reactants) == 0 or len(r.products) == 0:
        exchange_rxns.append(r.id)
        
exchange_rxns = list(set(exchange_rxns)-set(fatty_acid_rxns)-set(["Gln","Glu","HMR_9048"]))

change_rxns = ["HMR_9103","HMR_9438","HMR_9424","HMR_9078","HMR_9073","HMR_9216","HMR_9318","HMR_9354",
               "HMR_9142","HMR_9148","HMR_9386","HMR_9079","HMR_9730","HMR_9219","HMR_9086","HMR_9094",
               "HMR_9389","HMR_9088","HMR_9412","HMR_9047","HMR_9068",
               "HMR_9135","HMR_9310","HMR_9253","HMR_9254","HMR_9061","HMR_9259","HMR_9066","HMR_9062",
               "HMR_9070","HMR_m01580x","HMR_9201","HMR_9295","HMR_m01862x","HMR_9071","HMR_9063","HMR_9067",
               "HMR_9348","HMR_9038","HMR_9362","HMR_m02183x","HMR_m02439x","HMR_9042","HMR_9378","HMR_9087",
               "HMR_9043","HMR_9290","HMR_9133","HMR_9131","HMR_9069","HMR_9044","HMR_9423","HMR_9422","HMR_9426",
               "HMR_9045","HMR_9064","HMR_9437","HMR_9439","HMR_m03148x","HMR_9034"]

for reac_id in change_rxns: #exchange_rxns:
    plot_o2_pe(reac_id,o2_fluxes,growth_norm=growth_norm)
    plt.tight_layout()
    plt.savefig("../Figures/Hypoxia/{}_import_bmOFF{}_GlnOFF{}_FAmin{}_woBile.svg.svg".format(reac_id,biomass_off,glut_off,fatty_acid_min))
    plt.show()
    plt.close()

In [ ]:
change_rxns = ["HMR_9224","HMR_9084"] #"HMR_9284",
#"HMR_9074",

for reac_id in change_rxns: #exchange_rxns:
    plot_o2_pe(reac_id,o2_fluxes,growth_norm=growth_norm)
    plt.tight_layout()
    plt.savefig("../Figures/Hypoxia/{}_import_bmOFF{}_GlnOFF{}_FAmin{}_woBile.svg.svg".format(reac_id,biomass_off,glut_off,fatty_acid_min))
    plt.show()
    plt.close()